# Getting solution information

In this tutorial we will see how to extract information from an optimization model, such as solution information. Gurobi has organized this information in so called *attributes*. Each component of an optimization model, i.e., the [Model object](https://www.gurobi.com/documentation/9.0/refman/py_model.html), the [variables objects](https://www.gurobi.com/documentation/9.0/refman/py_var.html) and the [constraints objects](https://www.gurobi.com/documentation/9.0/refman/py_constr.html) are provided with a method called `getAttr` which can be used to query their specific attributes, corresponding to solution and model information. A central exercise is thus that of understanding which attributes can be queried from each object. The full list of attributes for each object is provided in the Gurobi documentation [here](https://www.gurobi.com/documentation/9.0/refman/attributes.html#sec:Attributes). This page should be kept at hand when working with optimization models.

## An example model

In this tutorial we will consider the following simple optimization problem.
(A model with many decision variables and constraints).

## Attributes of the Model

Let us start by having a look at some of the most important attributes of the `Model` object.
First of all, in the documentation you can see that the `Model` object has a [method `getAttr`](https://www.gurobi.com/documentation/9.0/refman/py_model_getattr.html). This method can be used to query information about the model as well as information about sets of decision variables and constraints. Let us see some examples.